In [2]:
!git clone https://github.com/wchill/HMP_Dataset.git

Cloning into 'HMP_Dataset'...
remote: Enumerating objects: 865, done.
remote: Total 865 (delta 0), reused 0 (delta 0), pack-reused 865
Receiving objects: 100% (865/865), 1010.96 KiB | 0 bytes/s, done.


In [3]:
! ls HMP_Dataset

Brush_teeth	Drink_glass  Getup_bed	  Pour_water	 Use_telephone
Climb_stairs	Eat_meat     impdata.py   README.txt	 Walk
Comb_hair	Eat_soup     Liedown_bed  Sitdown_chair
Descend_stairs	final.py     MANUAL.txt   Standup_chair


In [4]:
! ls HMP_Dataset/Brush_teeth

Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt
Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt
Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt
Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt
Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt
Accelerometer-2011-05-30-21-10-57-brush_teeth-f1.txt
Accelerometer-2011-05-30-21-55-04-brush_teeth-m2.txt
Accelerometer-2011-05-31-15-16-47-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-42-22-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-45-50-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-45-27-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-48-05-brush_teeth-f1.txt


In [6]:
!pip install pyspark

     |████████████████████████████████| 218.4MB 164kB/s  eta 0:00:01                    | 37.0MB 6.0MB/s eta 0:00:31
     |████████████████████████████████| 204kB 36.8MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/1e/5e/6a/17e906c94ec7246f260330a66e44a06a0809033ba2738a74a8
Successfully built pyspark


In [8]:
import os
#keep only datafile names
fileList = os.listdir("HMP_Dataset")
fileListFilter = [s for s in fileList if "_" in s]
fileListFilter

['Descend_stairs',
 'Climb_stairs',
 'Standup_chair',
 'Eat_meat',
 'Drink_glass',
 'Liedown_bed',
 'Brush_teeth',
 'Use_telephone',
 'Eat_soup',
 'Pour_water',
 'Getup_bed',
 'Sitdown_chair',
 'Comb_hair']

In [11]:
from pyspark.sql.types import StructType,StructField,IntegerType
#creating schema
schema = StructType([StructField("x",IntegerType(),True),
                    StructField("y",IntegerType(),True),
                    StructField("z",IntegerType(),True)])

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


In [19]:
from pyspark.sql.functions import lit
#for each file in fileListfitler 
df = None
for cat in fileListFilter:
    dataFile= os.listdir('HMP_Dataset/'+cat)
    
    #for each data in each file
    for data in dataFile:
#         print(data)
        
        #creating temp dataframe
        temp_df = spark.read.option("header","false").option("delimiter"," ").csv("HMP_Dataset/"+cat+'/'+data,schema=schema)

        #creating two new colums to know the data name(ex: accelerometer) and source names
        temp_df = temp_df.withColumn("class",lit(cat))
        temp_df = temp_df.withColumn("source",lit(data))
        
        #adding to df
        if df is None:
            #for first iteration
            df = temp_df
        else:
            #appending for every loop
            df = df.union(temp_df)
            

In [20]:
df.show()

+---+---+---+--------------+--------------------+
|  x|  y|  z|         class|              source|
+---+---+---+--------------+--------------------+
| 37| 48| 42|Descend_stairs|Accelerometer-201...|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|
| 39| 33| 40|Descend_stairs|Accelerometer-201...|
| 39| 43| 41|Descend_stairs|Accelerometer-201...|
| 39| 45| 41|Descend_stairs|Accelerometer-201...|
| 40| 44| 40|Descend_stairs|Accelerometer-201...|
| 41| 45| 40|Descend_stairs|Accelerometer-201...|
| 37| 52| 43|Descend_stairs|Accelerometer-201...|
| 36| 46| 40|Descend_stairs|Accelerometer-201...|
| 32| 49| 40|Descend_stairs|Accelerometer-201...|
| 28| 53| 43|Descend_stairs|Accelerometer-201...|
| 27| 44| 39|Descend_stairs|Accelerometer-201...|
| 21| 52| 43|Descend_stairs|Accelerometer-201...|
| 20| 47| 42|Descend_stairs|Accelerometer-201...|
| 21| 45| 42|Descend_stairs|Accelerometer-201...|
| 17| 47| 42|Descend_stairs|Accelerometer-201...|


In [23]:
#categorical values in categorical indices
from pyspark.ml.feature import StringIndexer

label = StringIndexer(inputCol="class",outputCol="classIndex")
labeled = label.fit(df).transform(df)
labeled.show()



+---+---+---+--------------+--------------------+----------+
|  x|  y|  z|         class|              source|classIndex|
+---+---+---+--------------+--------------------+----------+
| 37| 48| 42|Descend_stairs|Accelerometer-201...|       9.0|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|
| 39| 33| 40|Descend_stairs|Accelerometer-201...|       9.0|
| 39| 43| 41|Descend_stairs|Accelerometer-201...|       9.0|
| 39| 45| 41|Descend_stairs|Accelerometer-201...|       9.0|
| 40| 44| 40|Descend_stairs|Accelerometer-201...|       9.0|
| 41| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|
| 37| 52| 43|Descend_stairs|Accelerometer-201...|       9.0|
| 36| 46| 40|Descend_stairs|Accelerometer-201...|       9.0|
| 32| 49| 40|Descend_stairs|Accelerometer-201...|       9.0|
| 28| 53| 43|Descend_stairs|Accelerometer-201...|       9.0|
| 27| 44| 39|Descend_stairs|Accelerometer-201...|       9.0|
| 21| 52| 43|Descend_sta

In [27]:
#OneHotEncoder for catergorical indicies
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol='classIndex',outputCol="vectorCategory")
encoded = encoder.transform(labeled)

encoded.show()

+---+---+---+--------------+--------------------+----------+--------------+
|  x|  y|  z|         class|              source|classIndex|vectorCategory|
+---+---+---+--------------+--------------------+----------+--------------+
| 37| 48| 42|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 39| 33| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 39| 43| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 39| 45| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 40| 44| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 41| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 37| 52| 43|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 36| 46| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|
| 32| 49| 40

In [29]:
#input features to vectors

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

vector = VectorAssembler(inputCols=["x","y","z"],outputCol="features")
vectorised = vector.transform(encoded)

vectorised.show()

+---+---+---+--------------+--------------------+----------+--------------+----------------+
|  x|  y|  z|         class|              source|classIndex|vectorCategory|        features|
+---+---+---+--------------+--------------------+----------+--------------+----------------+
| 37| 48| 42|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[37.0,48.0,42.0]|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,40.0]|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,40.0]|
| 39| 33| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,33.0,40.0]|
| 39| 43| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,43.0,41.0]|
| 39| 45| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,41.0]|
| 40| 44| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[40.0,44.0,40.0]|
| 41| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1

In [32]:
#data normailzation

from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="features",outputCol="feature_Norm")

normalized_data = normalizer.transform(vectorised)

normalized_data.show()

+---+---+---+--------------+--------------------+----------+--------------+----------------+--------------------+
|  x|  y|  z|         class|              source|classIndex|vectorCategory|        features|        feature_Norm|
+---+---+---+--------------+--------------------+----------+--------------+----------------+--------------------+
| 37| 48| 42|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[37.0,48.0,42.0]|[0.50179006401732...|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,40.0]|[0.54366292339213...|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,40.0]|[0.54366292339213...|
| 39| 33| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,33.0,40.0]|[0.60106793320933...|
| 39| 43| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,43.0,41.0]|[0.54875175572205...|
| 39| 45| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,41

In [36]:
#pipeline

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[label,encoder,vector,normalizer])

model=pipeline.fit(df)
prediction = model.transform(df)

prediction.show()

+---+---+---+--------------+--------------------+----------+--------------+----------------+--------------------+
|  x|  y|  z|         class|              source|classIndex|vectorCategory|        features|        feature_Norm|
+---+---+---+--------------+--------------------+----------+--------------+----------------+--------------------+
| 37| 48| 42|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[37.0,48.0,42.0]|[0.50179006401732...|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,40.0]|[0.54366292339213...|
| 39| 45| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,40.0]|[0.54366292339213...|
| 39| 33| 40|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,33.0,40.0]|[0.60106793320933...|
| 39| 43| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,43.0,41.0]|[0.54875175572205...|
| 39| 45| 41|Descend_stairs|Accelerometer-201...|       9.0|(12,[9],[1.0])|[39.0,45.0,41

In [37]:
#retaining vectorCategory and feature_Norm for training model
df_train = prediction.drop('x').drop('y').drop('z').drop('class').drop('source').drop('features').drop("classIndex")
df_train.show()

+--------------+--------------------+
|vectorCategory|        feature_Norm|
+--------------+--------------------+
|(12,[9],[1.0])|[0.50179006401732...|
|(12,[9],[1.0])|[0.54366292339213...|
|(12,[9],[1.0])|[0.54366292339213...|
|(12,[9],[1.0])|[0.60106793320933...|
|(12,[9],[1.0])|[0.54875175572205...|
|(12,[9],[1.0])|[0.53943405072652...|
|(12,[9],[1.0])|[0.55814557218594...|
|(12,[9],[1.0])|[0.56285980242023...|
|(12,[9],[1.0])|[0.48080339161102...|
|(12,[9],[1.0])|[0.50850703969305...|
|(12,[9],[1.0])|[0.45142119414950...|
|(12,[9],[1.0])|[0.37955853580464...|
|(12,[9],[1.0])|[0.41731515072125...|
|(12,[9],[1.0])|[0.29716319953956...|
|(12,[9],[1.0])|[0.30244071561610...|
|(12,[9],[1.0])|[0.32288592281010...|
|(12,[9],[1.0])|[0.26040073078098...|
|(12,[9],[1.0])|[0.23383328952568...|
|(12,[9],[1.0])|[0.23464053310389...|
|(12,[9],[1.0])|[0.19064955134186...|
+--------------+--------------------+
only showing top 20 rows

